In [1]:
import os
import pandas as pd
import numpy as np

# List the folders inside `preprocessing` 

In [77]:
sorted([d for d in os.listdir() if os.path.isdir(d) and not d.startswith('.') and not d.startswith('test_') and not d.startswith('old')])

['211109_A01083_0095_BHNGMCDSXY',
 '211209_A01083_0102_BHGWFNDSX2',
 '220422_A01083_0130_AHNW7HDSX2',
 '220504_A01083_0133_BHVW3TDSX2',
 '220606_A01083_0137_AH2HYGDSX3',
 '220629_A01083_0139_AHYHKFDSX2',
 '220708_A01083_0142_AHCF75DSX3',
 '220728_A01083_0147_AHCC5YDSX3',
 '220819_A01083_0150_BHNWNGDSX3',
 '220830_IEO_PLIC',
 '220902_A01083_0155_AHHFNCDSX3',
 'GopalakrishnanV_2018',
 'IIGM_VC2',
 'PREDICT2_Zymo',
 'PREDICT2_Zymo__2',
 'run21_pool3',
 'run21_retrieve_reads',
 'run24_pool1']

In [3]:
samples_to_skip = set(['.ipynb_checkpoints', '.empty', 'Stats', 'Reports'])

In [104]:
# folder = '200313_A01083_0011_BH25HGDSXY'  # 5th run
# folder = '200327_A01083_0013_BH2752DSXY'  # 6th run
# folder = 'test_mismatch'                  # 6th run test "--mismatch 1"
# folder = '200409_A00302_0146_BHYGTCDSXX'  # 7th run
# folder = '200514_A01083_0014_AH2757DSXY'  # 8th run
# folder = '200522_A01083_0015_BH25WYDSXY'  # 9th run
# folder = '200616_A01083_0017_BHFLMWDSXY'  # 10th run
# folder = '200713_A01083_0019_AHFLW2DSXY'  # 11th run
# folder = 'PREDICT2_Zymo'
# folder = '200729_A01083_0027_BHFKJCDSXY'  # 12th run
# folder = 'PREDICT2_Zymo__2'
# folder = '200917_A01083_0034_AHYNCGDSXX'  # 13th run
# folder = '201103_A01083_0039_AHYNF2DSXX'  # 14th run
# folder = '201230_A01083_0046_BHFFTVDSXY'  # 15th run
# folder = '210121_A01083_0048_AHGVLMDSXY'  # 16th run / COVID UK Tim Spector
# folder = '210211_A01083_0052_AHVFKCDSXY'  # 17th run
# folder = '210226_A01083_0055_AHVCT5DSXY'  # 18th run
# folder = '210319_A01083_0057_BHN2CCDSXY'  # 19th run
# folder = '210510_A01083_0065_AHFC7NDSXY'  # 20th run
# folder = '210611_A01083_0066_BHN3GKDSXY'  # 21st run
# folder = '210723_A01083_0072_AHN3C5DSXY'  # 22nd run
# folder = '210902_A01083_0080_BHMV7MDSXY'  # 23rd run
# folder = '211029_A01083_0091_AHFJJTDSX2'  # 24th run
# folder = '211109_A01083_0095_BHNGMCDSXY'  # 25th run
# folder = '211209_A01083_0102_BHGWFNDSX2'  # 26th run
# folder = '220125_A01083_0115_AHTYJ7DSX2'  # 27th run
# folder = '220225_A01083_0119_AHGLGKDSXY'  # 28th run
# folder = '220325_A01083_0126_BHYN7NDSXY'  # 29th run
# folder = '/shares/CIBIO-Storage/CM/scratch/users/f.asnicar/ONCO_DUCTH_SWEDEN/reads/'  # ONCO_DUTCH_SWEDEN / Fredrik Boulund
# folder = '220422_A01083_0130_AHNW7HDSX2'  # SARDEGNA 1st run aprile 2022
# folder = '220504_A01083_0133_BHVW3TDSX2'  # 30th run
# folder = '220606_A01083_0137_AH2HYGDSX3'  # 31st run
# folder = '220629_A01083_0139_AHYHKFDSX2'  # 32nd run
# folder = '220708_A01083_0142_AHCF75DSX3'  # 33rd run
# folder = '220726_A01083_0146_BHCFHLDSX3'  # 34th run / 408 PLIC samples with split
# folder = '220728_A01083_0147_AHCC5YDSX3'  # SARDEGNA 2nd run luglio 2022
# folder = '220819_A01083_0150_BHNWNGDSX3'  # SARDEGNA 3rd run agosto 2022
# folder = '220902_A01083_0155_AHHFNCDSX3'  # 35th run / 4xx PLIC samples with split
# folder = '220830_IEO_PLIC_demux'  # IEO_PLIC
folder = '220921_A01083_0165_BHNMLCDSX3'  # 36th run

# Adapters contamination

In [ ]:
adapters = {}

for sample in os.listdir(folder):
    folder_sample = os.path.join(folder, sample)

    if (sample in samples_to_skip) or (not os.path.isdir(folder_sample)):
        continue

    adapters[sample] = 0.0
    n = 0

    for lane in os.listdir(os.path.join(folder_sample, 'fastqc_raw')):
        folder_sample_lane = os.path.join(folder_sample, 'fastqc_raw', lane)

        if not os.path.isdir(folder_sample_lane):
            continue

        save = False
        tmp_list = []

        with open(os.path.join(folder_sample_lane, 'fastqc_data.txt')) as f:
            for r in f:
                if r.startswith('>>Adapter Content'):
                    save = True

                if save and r.startswith('>>END_MODULE'):
                    save = False
                    break

                if save:
                    tmp_list.append(r.strip().split('\t'))

        adapters[sample] += sum(map(float, tmp_list[-1][1:]))
        n += 1

    adapters[sample] /= n

In [ ]:
len(adapters)

* 10 samples with **highest** % of adapters

In [ ]:
sorted(adapters.items(), key=lambda x: x[1], reverse=True)[:10]

* 10 samples with **lowest** % of adapters

In [ ]:
sorted(adapters.items(), key=lambda x: x[1], reverse=True)[-10:]

# Number of reads

In [99]:
nreads = {}

for sample in os.listdir(folder):
    folder_sample = os.path.join(folder, sample)

    if (sample in samples_to_skip) or (not os.path.isdir(folder_sample)):
        continue

    nreads[sample] = 0

    for lane in os.listdir(os.path.join(folder_sample, 'fastqc_raw')):
        folder_sample_lane = os.path.join(folder_sample, 'fastqc_raw', lane)
        
        if not os.path.isdir(folder_sample_lane):
            continue
        
        with open(os.path.join(folder_sample_lane, 'fastqc_data.txt')) as f:
            for r in f:
                if r.startswith('Total Sequences'):
                    nreads[sample] += int(r.strip().split('\t')[1])
                    break

In [100]:
len(nreads)

382

* 10 samples with **largest** number of reads

In [101]:
sorted(nreads.items(), key=lambda x: x[1], reverse=True)[:10]

[('PLIC_SHO_LN_117', 96682220),
 ('PLIC_SHO_LN_113', 92504196),
 ('PLIC_SHO_NS_28', 92036548),
 ('PLIC_SHO_LN_228', 91515004),
 ('PLIC_SHO_NS_67', 91111796),
 ('PLIC_SHO_LN_160', 85902184),
 ('PLIC_SHO_NS_26', 84851092),
 ('PLIC_SHO_NS_93', 84254958),
 ('PLIC_SHO_LN_140', 82336538),
 ('PLIC_SHO_LN_118', 82105732)]

* 10 samples with **lowest** number of reads

In [102]:
sorted(nreads.items(), key=lambda x: x[1], reverse=True)[-10:]

[('PLIC_SHO_LN_269', 9697038),
 ('PLIC_SHO_LN_273', 9366722),
 ('PLIC_SHO_LN_280', 9281630),
 ('PLIC_SHO_LN_271', 8996328),
 ('PLIC_SHO_LN_284', 8810970),
 ('PLIC_SHO_LN_270', 8622660),
 ('PLIC_SHO_LN_282', 8601930),
 ('PLIC_SHO_LN_289', 8268534),
 ('PLIC_SHO_LN_293', 6074736),
 ('PLIC_SHO_LN_272', 5321182)]

## Save `_adapters_nreads.tsv`

In [103]:
if not len(set(adapters.keys()) - set(nreads.keys())):
    print('writing {}_adapters_nreads.tsv'.format(folder))
    
    with open('{}_adapters_nreads.tsv'.format(folder), 'w') as f:
        f.write('#sample\tadapters%\tnreads\n')
        f.write('\n'.join(['{}\t{}\t{}'.format(sample, adapters[sample], nreads[sample]) for sample in nreads.keys()]) + '\n')
else:
    print('ERROR: adapters and nreads keys not identical!')

writing 220830_IEO_PLIC_demux_adapters_nreads.tsv


---

# Analysis for Zoe to evaluate the quality of the sequencing by Zymo

## Number of reads (raw) with Q>=30

In [12]:
qscore_raw = {}
allreads_raw = {}

for sample in os.listdir(folder):
    if not sample.startswith("15"):
        continue

    folder_sample = os.path.join(folder, sample)
    
    if (not os.path.isdir(folder_sample)) or (sample in ['.ipynb_checkpoints', '.empty', 'Stats', 'Reports']):
        continue

    qscore_raw[sample] = 0
    allreads_raw[sample] = 0
    
    for lane in os.listdir(os.path.join(folder_sample, 'fastqc_raw')):
        folder_sample_lane = os.path.join(folder_sample, 'fastqc_raw', lane)
        save = False

        if not os.path.isdir(folder_sample_lane):
            continue

        with open(os.path.join(folder_sample_lane, 'fastqc_data.txt')) as f:
            for r in f:
                if r.startswith('#'):
                    continue

                if r.startswith('>>Per sequence quality scores'):
                    save = True
                    continue
                    
                if save and r.startswith('>>END_MODULE'):
                    save = False
                    break
                    
                if save:
                    q = int(r.strip().split('\t')[0])
                    allreads_raw[sample] += int(float(r.strip().split('\t')[1]))
                    
                    if q >= 30:
                        qscore_raw[sample] += int(float(r.strip().split('\t')[1]))

## Number of reads (clean) with Q>=30

In [13]:
qscore_clean = {}
allreads_clean = {}

for sample in os.listdir(folder):
    if not sample.startswith("15"):
        continue

    folder_sample = os.path.join(folder, sample)
    
    if (not os.path.isdir(folder_sample)) or (sample in ['.ipynb_checkpoints', '.empty', 'Stats', 'Reports']):
        continue

    qscore_clean[sample] = 0
    allreads_clean[sample] = 0
    
    for lane in os.listdir(os.path.join(folder_sample, 'fastqc_clean')):
        folder_sample_lane = os.path.join(folder_sample, 'fastqc_clean', lane)
        save = False

        if not os.path.isdir(folder_sample_lane):
            continue

        with open(os.path.join(folder_sample_lane, 'fastqc_data.txt')) as f:
            for r in f:
                if r.startswith('#'):
                    continue

                if r.startswith('>>Per sequence quality scores'):
                    save = True
                    continue
                    
                if save and r.startswith('>>END_MODULE'):
                    save = False
                    break
                    
                if save:
                    q = int(r.strip().split('\t')[0])
                    allreads_clean[sample] += int(float(r.strip().split('\t')[1]))
                    
                    if q >= 30:
                        qscore_clean[sample] += int(float(r.strip().split('\t')[1]))

In [18]:
qscore = {'sample': [], 
          'Gbp raw': [], 
          'Gbp raw (Q>=30)': [], 
          'Gbp clean': [], 
          'Gbp clean (Q>=30)': [], 
          'Gbp diff': [], 
          'Gbp diff (Q>=30)': [], 
          'reads raw': [], 
          'reads raw (Q>=30)': [], 
          'reads clean': [], 
          'reads clean (Q>=30)': [], 
          'reads diff': [], 
          'reads diff (Q>=30)': [], 
          'raw (Q>=30)': [], 
          'clean (Q>=30)': []}

for s, q in qscore_clean.items():
    qscore['sample'].append(s)

    qscore['Gbp raw'].append((allreads_raw[s] * 150) / 1e9)
    qscore['Gbp clean'].append((allreads_clean[s] * 150) / 1e9)
    qscore['Gbp diff'].append(((allreads_raw[s] * 150) - (allreads_clean[s] * 150)) / 1e9)

    qscore['Gbp raw (Q>=30)'].append((qscore_raw[s] * 150) / 1e9)
    qscore['Gbp clean (Q>=30)'].append((q * 150) / 1e9)
    qscore['Gbp diff (Q>=30)'].append(((qscore_raw[s] * 150) - (q * 150)) / 1e9)

    qscore['reads raw'].append(allreads_raw[s])
    qscore['reads clean'].append(allreads_clean[s])
    qscore['reads diff'].append(allreads_raw[s] - allreads_clean[s])

    qscore['reads raw (Q>=30)'].append(qscore_raw[s])
    qscore['reads clean (Q>=30)'].append(q)
    qscore['reads diff (Q>=30)'].append(qscore_raw[s] - q)

    qscore['raw (Q>=30)'].append(qscore_raw[s] > 16333333)
    qscore['clean (Q>=30)'].append(q > 16333333)

qscore = pd.DataFrame.from_dict(qscore)
qscore.head()

,sample,Gbp raw,Gbp raw (Q>=30),Gbp clean,Gbp clean (Q>=30),Gbp diff,Gbp diff (Q>=30),reads raw,reads raw (Q>=30),reads clean,reads clean (Q>=30),reads diff,reads diff (Q>=30),raw (Q>=30),clean (Q>=30)
0,15937,5.820289,5.711621,5.788405,5.684299,0.031884,0.027321,38801928,38077471,38589369,37895328,212559,182143,True,True
1,15698,3.499858,3.416186,3.487805,3.406866,0.012052,0.009320,23332384,22774570,23252036,22712438,80348,62132,True,True
2,15757,4.139379,4.042826,4.124064,4.030694,0.015315,0.012132,27595858,26952173,27493760,26871292,102098,80881,True,True
3,15829,4.356426,4.268894,4.346661,4.262238,0.009765,0.006656,29042842,28459291,28977740,28414919,65102,44372,True,True
4,15843,11.099473,10.840722,11.057881,10.807932,0.041592,0.032790,73996486,72271478,73719206,72052877,277280,218601,True,True


In [19]:
qscore.describe()

,Gbp raw,Gbp raw (Q>=30),Gbp clean,Gbp clean (Q>=30),Gbp diff,Gbp diff (Q>=30),reads raw,reads raw (Q>=30),reads clean,reads clean (Q>=30),reads diff,reads diff (Q>=30)
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,3.100000e+02,3.100000e+02,3.100000e+02,3.100000e+02,3.100000e+02,3.100000e+02
mean,8.935281,8.718980,8.880085,8.671721,0.055196,0.047259,5.956854e+07,5.812654e+07,5.920057e+07,5.781148e+07,3.679754e+05,3.150606e+05
std,4.555286,4.440036,4.515397,4.404497,0.086843,0.082635,3.036857e+07,2.960024e+07,3.010265e+07,2.936332e+07,5.789535e+05,5.509005e+05
min,0.805035,0.795382,0.774250,0.765373,0.006591,0.002476,5.366898e+06,5.302548e+06,5.161665e+06,5.102488e+06,4.394000e+04,1.650500e+04
25%,5.431537,5.311542,5.386443,5.272271,0.022365,0.017638,3.621025e+07,3.541028e+07,3.590962e+07,3.514847e+07,1.490990e+05,1.175868e+05
50%,8.480339,8.270554,8.438704,8.239626,0.039512,0.031355,5.653560e+07,5.513703e+07,5.625803e+07,5.493084e+07,2.634130e+05,2.090315e+05
75%,11.689692,11.414266,11.644458,11.378603,0.059914,0.050696,7.793128e+07,7.609511e+07,7.762972e+07,7.585735e+07,3.994275e+05,3.379735e+05
max,26.255479,25.510253,26.157627,25.435732,1.259662,1.198051,1.750365e+08,1.700684e+08,1.743842e+08,1.695715e+08,8.397748e+06,7.987007e+06


## Sample(s) not passing the minimum quality threshold

In [22]:
qscore[qscore['clean (Q>=30)'] == False]['sample']

10     15889
31     15876
46     15689
65     15855
112    15873
123    15934
165    15723
175    15868
176    15894
178    15869
219    15726
269    15705
302    15871
Name: sample, dtype: object

---

# Check "Duplication"

In [41]:
dups = {}

for sample in os.listdir(folder):
    if not sample.startswith("15"):
        continue

    folder_sample = os.path.join(folder, sample)
    
    if (not os.path.isdir(folder_sample)) or (sample in ['.ipynb_checkpoints', '.empty', 'Stats', 'Reports']):
        continue

    dups[sample] = {'R1': 0, 'R2': 0, 'all': 0}

    for lane in os.listdir(os.path.join(folder_sample, 'fastqc_raw')):
        folder_sample_lane = os.path.join(folder_sample, 'fastqc_raw', lane)
        Rs = 'R1' if 'R1' in lane else 'R2' if 'R2' in lane else 'XX'
        
        if not os.path.isdir(folder_sample_lane):
            continue

        with open(os.path.join(folder_sample_lane, 'fastqc_data.txt')) as f:
            for r in f:
                if r.startswith('#Total Deduplicated Percentage'):
                    dups[sample][Rs] += 100. - float(r.strip().split('\t')[1])
                    dups[sample]['all'] += 100. - float(r.strip().split('\t')[1])
                    break

    dups[sample]['all'] /= len([lane for lane in os.listdir(os.path.join(folder_sample, 'fastqc_raw')) if os.path.isdir(os.path.join(folder_sample, 'fastqc_raw', lane))])

In [43]:
dups_df = pd.DataFrame.from_dict(dups).T

In [50]:
dups_df[['all']].describe()

,all
count,282.000000
mean,17.117959
std,9.540601
min,3.128253
25%,9.784817
50%,14.801356
75%,22.691174
max,66.969859


In [49]:
dups_df[dups_df['all'] > 40][['all']]

,all
80051_31,43.753384
83581_31,45.552779
83582_31,48.237725
83583_31,52.487036
80190_31,47.971683
80118_31,66.969859


# 13th run ONCOBIOME read stats

In [75]:
onco = {'BLADDER' : [], 'IGR' : [], 'MM' : [], 'ORL' : [], 'Pr' : [], 'SK' : [], 'UM' : []}

for prj in onco.keys():
    for sample in os.listdir('200917_A01083_0034_AHYNCGDSXX/'):
        if prj not in sample:
            continue

        read_stats = [f for f in os.listdir('200917_A01083_0034_AHYNCGDSXX/' + sample) if f.endswith('_summary.stats')][0]

        with open('200917_A01083_0034_AHYNCGDSXX/{}/{}'.format(sample, read_stats)) as f:
            onco[prj].append(int(f.readlines()[-1].split('\t')[2]))

In [78]:
for prj, reads in onco.items():
    print(prj, sum(reads), np.mean(reads))

BLADDER 3057144043 87346972.65714286
IGR 5711107763 80438137.50704226
MM 2112776799 31070247.04411765
ORL 1248126022 73419177.76470588
Pr 5108999039 83754082.60655738
SK 3016680164 86190861.82857142
UM 48374742 16124914.0


# CM_CELIACHIA (NCGS, CDIS, CNTR) read stats (13th run)

In [93]:
celiachia_prefixes = ['NCGS', 'CDIS', 'CNTR']
celiachia_reads = []
fld = '/shares/CIBIO-Storage/CM/scratch/data/meta/CM_CELIACHIA/reads/'

for sample in os.listdir(fld):
    if len([i for i in celiachia_prefixes if i in sample]) == 0:
        continue

    read_stats = [f for f in os.listdir(fld + sample) if f.endswith('_summary.stats')][0]

    with open(fld + '{}/{}'.format(sample, read_stats)) as f:
        celiachia_reads.append(int(f.readlines()[-1].split('\t')[2]))

In [94]:
len(celiachia_reads), sum(celiachia_reads), np.mean(celiachia_reads)

(53, 1545506894, 29160507.433962263)

# 22nd run adapters contamination problem

In [63]:
folder = '210723_A01083_0072_AHN3C5DSXY'  # 22nd run
adapters_22nd = {}

for sample in os.listdir(folder):
    folder_sample = os.path.join(folder, sample)

    if (not os.path.isdir(folder_sample)) or (sample in ['.ipynb_checkpoints', '.empty', 'Stats', 'Reports']):
        continue

    if not os.path.isdir(os.path.join(folder_sample, 'fastqc_clean')):
        continue

    adapters_22nd[sample] = 0.0
    n = 0

    for lane in os.listdir(os.path.join(folder_sample, 'fastqc_clean')):
        folder_sample_lane = os.path.join(folder_sample, 'fastqc_clean', lane)

        if not os.path.isdir(folder_sample_lane):
            continue

        save = False
        tmp_list = []

        with open(os.path.join(folder_sample_lane, 'fastqc_data.txt')) as f:
            for r in f:
                if r.startswith('>>Adapter Content'):
                    save = True
                    continue

                if save and r.startswith('>>END_MODULE'):
                    save = False
                    break

                if save:
                    tmp_list.append(r.strip().split('\t'))

        try:
            adapters_22nd[sample] += sum(map(float, tmp_list[-1][1:]))
        except:
            adapters_22nd[sample] += 1000  # there are no values in the FastQC report

        n += 1

    adapters_22nd[sample] /= n

In [64]:
len(adapters_22nd)

355

In [65]:
# 10 samples with highest % of adapters
sorted(adapters_22nd.items(), key=lambda x: x[1], reverse=True)[:10]

[('BP29030', 1000.0),
 ('ONCOCBC_TF-1361-3_STO-001', 0.01378371128190367),
 ('ONCOCBC_TF-2306-7_STO-001', 0.011435705585839083),
 ('ONCOCBC_TF-1176-9_STO-001', 0.009434046408823697),
 ('TEAG-WP4-TGS04-SNF', 0.008650616411142153),
 ('TEAG-WP4-TGS06-1', 0.008437612422198267),
 ('ONCOCBC_TF-1653-3_STO-001', 0.008201090003385229),
 ('TEAG-WP4-TGS06-10', 0.008166360091530454),
 ('TEAG-WP4-TGS08-6', 0.007980372925013413),
 ('ONCOCBC_TF-2633-5_STO-001', 0.007789539845908094)]